In [8]:
# CUDA 12.1
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121

!pip install torch_geometric
!pip install torch-geometric-temporal
!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.4.0+cu121.html

# Optional dependencies:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
!pip install networkx numpy==1.24.3


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.9 MB 4.2 MB/s eta 0:00:03
   ------------- -------------------------- 4.5/12.9 MB 9.9 MB/s eta 0:00:01
   ---------------------------- ----------- 9.2/12.9 MB 14.3 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 15.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ----- ---------------------------------- 1.8/12.9 MB 8.4 MB/s eta 0:00:02
     ----------------- ---------------------- 5.5/12.9 MB 13.4 MB/s eta 0:00:01
     -------------------------- ------------- 8.4/12.9 MB 14.4 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.9 MB 14.2 MB/s eta 0:00:01
     --------------------------------------- 12.9/12.9 MB 14.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
     ---- ----------------------------------- 1.8/14.8 MB 8.4 MB/s eta 0:00:02
     -------------- ------------------------- 5.5/14.8 MB 12.9 MB/s eta 0:00:01
     ---------------------- ----------------- 8.4/14.8 MB 14.1 MB/s eta 0:00:01
     ------------------------------- ------- 12.1/14.8 MB 14.5 MB/s eta 0:00:01
     --------------------------------------- 14.8/14.8 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GConvGRU
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal

# reproducible
np.random.seed(42)
torch.manual_seed(42)

def generate_dynamic_graph(num_nodes=10, num_timesteps=8):
    edge_indices, edge_weights, features, targets = [], [], [], []
    for t in range(num_timesteps):
        g = nx.erdos_renyi_graph(num_nodes, p=0.4, seed=t)
        g.add_nodes_from(range(num_nodes))  # ensure nodes 0..num_nodes-1

        edges = list(g.edges())
        if len(edges) == 0:
            edge_index = np.empty((2, 0), dtype=np.int64)
            edge_attr = np.empty((0, 2), dtype=np.float32)
        else:
            edge_index = np.array(edges, dtype=np.int64).T  # shape (2, E)
            edge_attr = np.random.rand(edge_index.shape[1], 2).astype(np.float32)

        x = np.random.rand(num_nodes, 3).astype(np.float32)
        y = (edge_attr.sum(axis=1) > 1.0).astype(np.int64) if edge_attr.shape[0] > 0 else np.empty((0,), dtype=np.int64)
        w = edge_attr[:, 0].astype(np.float32) if edge_attr.shape[0] > 0 else np.empty((0,), dtype=np.float32)

        edge_indices.append(edge_index)
        edge_weights.append(w)
        features.append(x)
        targets.append(y)

    return DynamicGraphTemporalSignal(edge_indices, edge_weights, features, targets)


# create dataset
num_nodes = 10
dataset = generate_dynamic_graph(num_nodes=num_nodes, num_timesteps=8)
print("Snapshots:", dataset.snapshot_count)

# model
class RecurrentLinkClassifier(torch.nn.Module):
    def __init__(self, node_features, hidden_dim, num_nodes):
        super().__init__()
        self.num_nodes = num_nodes
        self.recurrent = GConvGRU(node_features, hidden_dim, K=2)
        self.fc = torch.nn.Linear(hidden_dim * 2, 2)

    def forward(self, x, edge_index, h=None):
        # ensure h has correct shape [num_nodes, hidden_dim]
        if h is None or h.size(0) != self.num_nodes:
            h = torch.zeros(self.num_nodes, self.recurrent.out_channels, device=x.device)
        # IMPORTANT: pass hidden state as keyword H=h (NOT as third positional argument)
        h = self.recurrent(x, edge_index, edge_weight=None, H=h)
        return h

    def predict_links(self, h, edge_index):
        src, dst = edge_index
        # safety check
        if src.max() >= h.size(0) or dst.max() >= h.size(0):
            raise ValueError(f"edge_index out of range: node count={h.size(0)}, max src={src.max()}, max dst={dst.max()}")
        z = torch.cat([h[src], h[dst]], dim=1)
        return self.fc(z)


device = torch.device("cuda")
model = RecurrentLinkClassifier(node_features=3, hidden_dim=16, num_nodes=num_nodes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def to_tensor(data, dtype=torch.float32):
    if isinstance(data, np.ndarray):
        return torch.from_numpy(data).to(dtype=dtype, device=device)
    elif isinstance(data, torch.Tensor):
        return data.to(dtype=dtype, device=device)
    else:
        raise TypeError(f"Unexpected data type: {type(data)}")

# training
epochs = 200
for epoch in range(epochs):
    print(f"Epoch {epoch+1:02d}/{epochs}")
    model.train()
    total_loss = 0.0
    h = None
    for snapshot in dataset:
        x = to_tensor(snapshot.x, torch.float32)               # [num_nodes, feat]
        edge_index = to_tensor(snapshot.edge_index, torch.long)  # [2, E]
        if edge_index.shape[1] == 0:
            continue
        # snapshot.y is 1D array of length E
        y_np = snapshot.y
        y = torch.from_numpy(y_np).long().to(device) if isinstance(y_np, np.ndarray) else y_np.long().to(device)

        # FORWARD - pass H as keyword to avoid being misinterpreted as edge_weight
        h = model(x, edge_index, h)
        h = h.detach()
        preds = model.predict_links(h, edge_index)  # [E, 2]

        loss = criterion(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += float(loss.item())

    print(f"Epoch {epoch+1:02d} | Loss: {total_loss:.4f}")

# evaluation
model.eval()
h = None
correct, total = 0, 0
with torch.no_grad():
    for snapshot in dataset:
        x = to_tensor(snapshot.x, torch.float32)
        edge_index = to_tensor(snapshot.edge_index, torch.long)
        if edge_index.shape[1] == 0:
            continue
        y_np = snapshot.y
        y = torch.from_numpy(y_np).long().to(device) if isinstance(y_np, np.ndarray) else y_np.long().to(device)

        h = model(x, edge_index, h)
        preds = model.predict_links(h, edge_index)
        pred_labels = preds.argmax(dim=1)
        correct += (pred_labels == y).sum().item()
        total += y.size(0)

if total > 0:
    print(f"Accuracy: {correct/total:.4f} ({correct}/{total})")
else:
    print("No edges to evaluate.")


Snapshots: 8
Epoch 01/200
Epoch 01 | Loss: 5.7364
Epoch 02/200
Epoch 02 | Loss: 5.5187
Epoch 03/200
Epoch 03 | Loss: 5.5395
Epoch 04/200
Epoch 04 | Loss: 5.4838
Epoch 05/200
Epoch 05 | Loss: 5.4420
Epoch 06/200
Epoch 06 | Loss: 5.4163
Epoch 07/200
Epoch 07 | Loss: 5.4041
Epoch 08/200
Epoch 08 | Loss: 5.3886
Epoch 09/200
Epoch 09 | Loss: 5.3730
Epoch 10/200
Epoch 10 | Loss: 5.3593
Epoch 11/200
Epoch 11 | Loss: 5.3477
Epoch 12/200
Epoch 12 | Loss: 5.3368
Epoch 13/200
Epoch 13 | Loss: 5.3263
Epoch 14/200
Epoch 14 | Loss: 5.3164
Epoch 15/200
Epoch 15 | Loss: 5.3071
Epoch 16/200
Epoch 16 | Loss: 5.2983
Epoch 17/200
Epoch 17 | Loss: 5.2899
Epoch 18/200
Epoch 18 | Loss: 5.2818
Epoch 19/200
Epoch 19 | Loss: 5.2740
Epoch 20/200
Epoch 20 | Loss: 5.2665
Epoch 21/200
Epoch 21 | Loss: 5.2592
Epoch 22/200
Epoch 22 | Loss: 5.2521
Epoch 23/200
Epoch 23 | Loss: 5.2453
Epoch 24/200
Epoch 24 | Loss: 5.2386
Epoch 25/200
Epoch 25 | Loss: 5.2320
Epoch 26/200
Epoch 26 | Loss: 5.2257
Epoch 27/200
Epoch 27 | L